In [1]:
#Reference:
#https://www.novixys.com/blog/using-aes-encryption-decryption-python-pycrypto/#4_Encrypting_with_AES
#https://stackoverflow.com/questions/36834580/iv-must-be-16-bytes-long-error-in-aes-encryption
#https://pycryptodome.readthedocs.io/en/latest/src/util/util.html

In [2]:
import socket 
import random
from Crypto.Cipher import AES
from base64 import b64encode
from base64 import b64decode
import os

In [3]:
# bind all IP
HOST = '172.20.10.3'
# Listen on Port 
PORT = 44444
#Size of receive buffer   
BUFFER_SIZE = 1024    
# Create a TCP/IP socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# Bind the socket to the host and port
s.bind(('0.0.0.0', PORT))

In [4]:
from Crypto.Cipher import AES
from base64 import b64encode, b64decode
from Crypto.Util.Padding import pad, unpad

BLOCK_SIZE = 16

def encrypt(plaintext, key, IV):
    cipher = AES.new(key, AES.MODE_CBC, IV)
    cipher_bytes = cipher.encrypt(pad(b64encode(plaintext), BLOCK_SIZE))
    return b64encode(cipher_bytes)

def decrypt(ct, passkey, IV):
    cipher = AES.new(passkey, AES.MODE_CBC, IV)
    decrypted = cipher.decrypt(b64decode(ct))
    return(unpad(decrypted, BLOCK_SIZE))


In [5]:
# import base64
# from Cryptodome.Cipher import AES
# from Cryptodome.Random import get_random_bytes

# key = hashlib.sha256(b'16-character key').digest()

# def encrypt(raw):
#     BS = AES.block_size
#     pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)

#     raw = base64.b64encode(pad(raw).encode('utf8'))
#     iv = get_random_bytes(AES.block_size)
#     cipher = AES.new(key, mode= AES.MODE_CFB,iv= iv)
#     print(iv)
#     return base64.b64encode(iv + cipher.encrypt(raw))

# def decrypt(enc):
#     unpad = lambda s: s[:-ord(s[-1:])]

#     enc = base64.b64decode(enc)
#     iv = enc[:AES.block_size]
#     cipher = AES.new(key, AES.MODE_CFB, iv)
#     return unpad(base64.b64decode(cipher.decrypt(enc[AES.block_size:])).decode('utf8'))

In [6]:
#AES requires 16 byte key
#In addition to the key, AES also needs an initialization vector. This initialization vector is generated with every encryption, and its purpose is to produce different encrypted data so that an attacker cannot use cryptanalysis to infer key data or message data.
#A 16-byte initialization vector is required which is generated as follows.
# def aes_encrypt(plaintext):
#     #iv = os.urandom(16)
#     print(iv)
#     aes_mode = AES.MODE_CBC
#     obj = AES.new(key, aes_mode, iv)
#     ciphertext = obj.encrypt(plaintext)
#     return ciphertext


# def aes_encrypt(plaintext):
#     #iv = os.urandom(16)
#     #Encrypt and Base-64 encode
#     aes_mode = AES.MODE_CBC
#     cipher = AES.new(key, aes_mode, iv)
#     return b64encode(iv + cipher.encrypt(plaintext))
# #     return cipher.encrypt(plaintext)

In [7]:
# print(s.sendto("Request for connection".encode('utf-8'), (HOST, PORT)))
# # #Send Authentication
# data, addr = s.recvfrom(1024)
# # for i in range(0, len(data), 2):
# #     data += data[i:i+2] + " " 
# print(data)

In [8]:
# while True:
#Request for authentication
s.sendto("Request for connection".encode('utf-8'), (HOST, PORT))
print("Python to Arduino: Request for connection")
# #Send Authentication
data, addr = s.recvfrom(PORT)
print("Arduino to Python(iv): ", data)

#Send username in cleartext and password in encrypted text separated by comma
username=bytes("Alice", 'ascii')
password="Happyme11!"
plaintext = bytes(password, 'ascii') # did I convert these bytes correctly from the sample code?
passkey = bytes("1111111111111111", 'ascii') #have I converted the passkey from the example code correctly?
# iv = b64encode(os.urandom(16)).decode('utf-8')
IV = data
IV=IV[:len(IV)-1] # have I converted the IV from the example code correctly
encrypted_bytes = encrypt(plaintext, passkey, IV)
print("Python to Arduino (Cipher text): ", encrypted_bytes)
# s.sendto(encrypted_bytes, (HOST, PORT))
# print(type(encrypted_bytes))
# s.sendto(bytes(username+";"+str(encrypted_bytes), "ascii"), (HOST, PORT))
# s.sendto((username+";"+str(encrypted_bytes)).encode('utf-8'), (HOST, PORT))
# print(type((username+";"+str(encrypted_bytes)).encode('utf-8')))
s.sendto(username+b";"+encrypted_bytes, (HOST, PORT))
message, addr = s.recvfrom(PORT)
message, addr = s.recvfrom(PORT)
print("Arduino to python (Acknowledgement): ", str(message.decode('utf-8')))

for i in range(3):
    s.sendto("BLINK_LED_A AND BLINK_ON_LED_B".encode('utf-8'), (HOST, PORT))
    print("Python to Arduino: BLINK_LED_A AND BLINK_ON_LED_B")




Python to Arduino: Request for connection
Arduino to Python(iv):  b'\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\xca'
Python to Arduino (Cipher text):  b'r7WU30WP5hj/2JKGVrq9pZrffhLoadf/EiG2RYAWP9o='
Arduino to python (Acknowledgement):  CONNECTED
Python to Arduino: BLINK_LED_A AND BLINK_ON_LED_B
Python to Arduino: BLINK_LED_A AND BLINK_ON_LED_B
Python to Arduino: BLINK_LED_A AND BLINK_ON_LED_B


In [9]:
# data=data.decode('unicode_escape')
# data=data.encode('utf-8')

# data='0x'.join(a+b for a,b in zip(data[::2], data[1::2]))
# data='0x'+data
# print(data)

# enc_iv=""
# for i in range(0, len(data1), 4):
#     code= data1[i:i+4]
#     hex_string = code[2:]
#     bytes_object = bytes.fromhex(hex_string)
# #     ascii_string = bytes_object.decode("ASCII")
# #     enc_iv+=ascii_string
#     print (bytex_object)

In [10]:
# hex_string = "0x65"[2:]
# bytes_object = bytes.fromhex(hex_string)
# ascii_string = bytes_object.decode("ASCII")
# print(ascii_string)

In [11]:
# plaintext = bytes("chimipassword@12", 'ascii') # did I convert these bytes correctly from the sample code?
# passkey = bytes("1111111111111111", 'ascii') #have I converted the passkey from the example code correctly?
# # iv = b64encode(os.urandom(16)).decode('utf-8')
# IV = data # have I converted the IV from the example code correctly?

# encrypted_bytes = encrypt(plaintext, passkey, IV)

# s.close()

In [12]:
# print(encrypted_bytes)

In [13]:
# Decryption
# print(b64decode(decrypt(encrypted_bytes, passkey, IV)))
# print(s.sendto("Request for connection".encode('utf-8'), (HOST, PORT)))


In [14]:
# while True: 
#     print(s.sendto("Request for connection".encode('utf-8'), (HOST, PORT)))
#     data, iv = s.recvfrom(1024)
#     print(data, iv)
#     s.close()
#     False
    

In [15]:
# while True:
#     s.sendto(encrypted_bytes, (HOST, PORT))

In [16]:
4s.close()

SyntaxError: invalid syntax (<ipython-input-16-32b0e3b2fd82>, line 1)